In [ ]:
from ppget.source import ModelSource
import matplotlib as mplt
mplt.rcParams["figure.figsize"] = (20,5)

start_date = "2007-07-01"
end_date = "2007-07-08"
feature = "Global_active_power"

source = ModelSource.load(start_date, end_date, feature)
source.missing_data()

In [ ]:
series = source.series()
series.head()